#Imports

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision
import sys
import os
import pandas as pd
from skimage import io
from google.colab import drive
from PIL import Image

In [52]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [53]:
root_dir = '/content/gdrive/MyDrive/cats_and_dogs'

In [54]:
class CatsAndDogsDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform;
  
  def __len__(self):
    return len(self.annotations)
  
  def __getitem__(self, index):
    img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])
    image =  Image.open(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index,1]))
    if self.transform:
      image = self.transform(image)
    return (image, y_label)

In [55]:
my_transforms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop((224,224)),
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.0,0.0,0.0], std=[1.0,1.0,1.0])
])

In [56]:
dataset = CatsAndDogsDataset(csv_file=root_dir+'/cats_dogs.csv',root_dir=root_dir+'/cats_dogs_resized', transform=my_transforms)

In [57]:
img_num = 0
for _ in range(10):
  for img, label in dataset:
    save_image(img,'img'+str(img_num)+'.png')
    img_num+=1